## How to run your customized workflow?

In addition to our automatically generated workflow, you also have the option to create a unique workflow tailored to your own ideas. Herein, you have the capacity to delineate the specific content of each subtask and articulate the description of the agent responsible for its execution. Concurrently, you possess the ability to stipulate the interdependencies among subtasks, thereby orchestrating the sequence in which the workflow is executed.

### Requirement
You need to have an OpenAI API Key.  
Notice that our default model is gpt4omini, please feel free to replace it with the model you wish to use in the config.py file.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"

### Import necessary module

In [2]:
import asyncio
import nest_asyncio
import json
from flow import Flow
from summary import Summary
from workflow import Task, Workflow

### Set overall task & Customize your own workflow
Firstly, you need to define your overall task. After that, you can freely define your own workflow.  
Following attributes need to be provided for each subtask:
- **`id`**: The identifier for this subtask.  
  Format: `"taskx"`, where `x` is an Arabic numeral (e.g., `task1`, `task2`).

- **`objective`**: The specific content or goal of this subtask.

- **`agent_id`**: The identifier for the agent responsible for executing this subtask.

- **`next`**: A list of tasks that depend on the completion of this subtask.  
  Values in this list must be `id`s of previously defined tasks.

- **`prev`**: A list of tasks that this subtask depends on.  
  Values in this list must be `id`s of previously defined tasks.

- **`agent`**: The name of the agent responsible for completing this subtask.  
  The same `agent_id` must always correspond to the same `agent`.

Note That：
- You are only permitted to modify the contents within the `overall_task` and `tasks_data` sections.
- Your workflow **must not** contain any form of circular dependencies.
- Ensure that all `next` and `prev` values reference valid `id`s of previously defined tasks.
- The `agent_id` and `agent` must remain consistent across all tasks.

Here, we provide an illustrative example of creating a simple Rock-Paper-Scissors game, you may observe the execution results of our example at the very bottom of this notebook. Now, proceed to craft your own workflow in accordance with your conceptual vision!

In [3]:
# Customize your overall task here.
overall_task: str = '''Develop a Rock-Paper-Scissors game with a graphical user interface (GUI) in Python. The game should allow a player to compete against a naive AI that randomly selects Rock, Paper, or Scissors. The UI should display the player’s choice, the AI’s choice, and the game result (win, lose, or draw). Provide an interactive and user-friendly experience.'''

# Use a list of dictionaries to initialize task data here.
tasks_data = [
    {
        "id": "task0",
        "objective": "Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.",
        "agent_id": 0,
        "next": ["task2"],
        "prev": [],
        "agent": "UI Designer",
    },
    {
        "id": "task1",
        "objective": "Design a naive AI and core game logic to determine the winner based on player and AI choices in python.",
        "agent_id": 1,
        "next": ["task2"],
        "prev": [],
        "agent": "Game Logic Developer",
    },
    {
        "id": "task2",
        "objective": "Combine the UI and game logic, test the game, and prepare it for deployment.",
        "agent_id": 2,
        "next": [],
        "prev": ["task0", "task1"],
        "agent": "Integration Specialist",
    },
]

# Create the workflow
workflow = Workflow(tasks={task_data["id"]: Task(**task_data) for task_data in tasks_data})

### Run your workflow

In [ ]:
manager = Flow(overall_task=overall_task, refine_threhold=0, max_refine_itt=0 n_candidate_graphs=0, workflow=workflow)
nest_asyncio.apply()
asyncio.run(manager.run_async())

2025-02-25 16:07:03 [INFO] Executing task 'Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.' with context: No completed previous tasks context available....
2025-02-25 16:07:03 [INFO] Task 'Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.' started by agent '0'.
2025-02-25 16:07:03 [INFO] Executing task 'Design a naive AI and core game logic to determine the winner based on player and AI choices in python.' with context: No completed previous tasks context available....
2025-02-25 16:07:03 [INFO] Task 'Design a naive AI and core game logic to determine the winner based on player and AI choices in python.' started by agent '1'.


object: Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.
object: Design a naive AI and core game logic to determine the winner based on player and AI choices in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.


2025-02-25 16:07:11 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 16:07:11 [INFO] Task task1 completed with result: To design a naive AI and core game logic for a Rock-Paper-Scissors game in Python, the following code is provided. This code includes the implementation of the AI, the logic to determine the winner based on the player’s and AI’s choices, and the necessary functions to facilitate gameplay.

```python
import random

def naive_ai_choice():
    """
    Function to simulate the naive AI's choice.
    The AI randomly selects Rock, Paper, or Scissors.
    """
    choices = ['Rock', 'Paper', 'Scissors']
    return random.choice(choices)

def determine_winner(player_choice, ai_choice):
    """
    Function to determine the winner of the game based on the choices made by the player and the AI.
    
    Parameters:
    player_choice (str): The choice made by the player ('Rock', 'Paper', or 'Scissors').
    ai_choice (str): The choi

1 inf


2025-02-25 16:07:11 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 16:07:11 [INFO] Task task0 completed with result: To design and implement the user interface for a Rock-Paper-Scissors game using the tkinter library in Python, the following code creates a simple GUI. This interface includes buttons for user input, displays for the computer's choice and the game result, and is structured to provide an interactive experience.

```python
import tkinter as tk
import random

class RockPaperScissorsGame:
    def __init__(self, master):
        self.master = master
        master.title("Rock-Paper-Scissors Game")

        self.label = tk.Label(master, text="Choose Rock, Paper, or Scissors:")
        self.label.pack()

        self.rock_button = tk.Button(master, text="Rock", command=lambda: self.play('Rock'))
        self.rock_button.pack(side=tk.LEFT)

        self.paper_button = tk.Button(master, text="Paper", command=lambda: self.play('Pa

2 inf
object: Combine the UI and game logic, test the game, and prepare it for deployment.
next: No downstream objectives available.


2025-02-25 16:07:28 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 16:07:28 [INFO] Task task2 completed with result: To combine the user interface and game logic for the Rock-Paper-Scissors game, the following Python code integrates the tkinter GUI with the naive AI and the game logic. This implementation ensures that the player can interact with the UI, make choices, and see the results of their game against the AI. The combined solution is designed to provide a seamless experience for the user.

```python
import tkinter as tk
import random

class RockPaperScissorsGame:
    def __init__(self, master):
        self.master = master
        master.title("Rock-Paper-Scissors Game")

        self.label = tk.Label(master, text="Choose Rock, Paper, or Scissors:")
        self.label.pack()

        self.rock_button = tk.Button(master, text="Rock", command=lambda: self.play('Rock'))
        self.rock_button.pack(side=tk.LEFT)

        self.pap

3 inf


### Store necessary information and summary the output

In [5]:
workflow_data = {
    tid: task.__dict__ for tid, task in manager.workflow.tasks.items()
}
# Store the final workflow
with open('manually_result.json', 'w', encoding='utf-8') as file:
    json.dump(workflow_data, file, indent=4)

summary = Summary()

# Generate and save a summary of the workflow results
chat_result = summary.summary(overall_task, workflow_data)
with open("example.txt", "w", encoding="utf-8") as file:
    file.write(chat_result)

2025-02-25 16:07:44 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The following Python code represents the outcome generated by our FLOW, based on the previously defined workflow.

In [6]:
import tkinter as tk
import random

def get_ai_choice():
    """A simple AI that randomly selects Rock, Paper, or Scissors."""
    return random.choice(['Rock', 'Paper', 'Scissors'])

def determine_winner(player_choice, ai_choice):
    """Core game logic to determine the winner based on player and AI choices."""
    if player_choice == ai_choice:
        return 'Draw'
    elif (player_choice == 'Rock' and ai_choice == 'Scissors') or \
         (player_choice == 'Paper' and ai_choice == 'Rock') or \
         (player_choice == 'Scissors' and ai_choice == 'Paper'):
        return 'You Win'
    else:
        return 'You Lose'

class RockPaperScissors:
    def __init__(self, master):
        self.master = master
        master.title('Rock-Paper-Scissors')

        self.player_choice = None
        self.ai_choice = None
        self.result = None

        self.label = tk.Label(master, text='Choose Rock, Paper, or Scissors:', font=('Arial', 14))
        self.label.pack()

        self.rock_button = tk.Button(master, text='Rock', command=lambda: self.play('Rock'), font=('Arial', 12))
        self.rock_button.pack(pady=5)

        self.paper_button = tk.Button(master, text='Paper', command=lambda: self.play('Paper'), font=('Arial', 12))
        self.paper_button.pack(pady=5)

        self.scissors_button = tk.Button(master, text='Scissors', command=lambda: self.play('Scissors'), font=('Arial', 12))
        self.scissors_button.pack(pady=5)

        self.result_label = tk.Label(master, text='', font=('Arial', 14))
        self.result_label.pack(pady=20)

    def play(self, player_choice):
        self.player_choice = player_choice
        self.ai_choice = get_ai_choice()
        self.result = determine_winner(self.player_choice, self.ai_choice)
        self.display_result()

    def display_result(self):
        self.result_label.config(text=f'You chose: {self.player_choice}\nAI chose: {self.ai_choice}\nResult: {self.result}')

if __name__ == '__main__':
    root = tk.Tk()
    game = RockPaperScissors(root)
    root.mainloop()